In [114]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [115]:
def extract_id_from_row(tr):
    # Find all <td> elements in the row
    td_elements = tr.find_all('td')
    
    # Check if there are at least two <td> elements (we expect the player name in the second <td>)
    if len(td_elements) > 1:
        a_tag = td_elements[0].find('a', href=True)
        if a_tag:
            # Extract the player ID from the href attribute
            href = a_tag['href']
            player_id = href.split('/')[3]  # Assuming the ID is in the 4th position
            
            # Remove the .shtml extension
            if player_id.endswith('.shtml'):
                player_id = player_id[:-6]
            
            return player_id
    return None

In [116]:
marcel_df = []

base_url = "https://www.baseball-reference.com/leagues/majors/{year}-projections.shtml"

for year in range(2021, 2025):
    stats_url = base_url.format(year=year)
    response = requests.get(stats_url)
    soup = BeautifulSoup(response.text, "html.parser")
    tables = soup.find_all("table")
    table = tables[0]

    data = []
    for tr in table.find_all('tr'):
        row = []
        for td in tr.find_all(['th', 'td']):
            row.append(td.text.strip())
        player_id = extract_id_from_row(tr)
        row.append(player_id)
        if row:
                data.append(row)

    headers = data[0]
    df = pd.DataFrame(data[1:], columns=headers)
    df.rename(columns={df.columns[-1]: 'key_bbref'}, inplace=True)
    df = df[['key_bbref'] + list(df.columns[:-1])]
    df.drop(columns=df.columns[1], inplace=True)
    df.insert(2, "Season", year + 1)    

    marcel_df.append(df)

marcel_df = pd.concat(marcel_df, ignore_index=True)

In [117]:
comparison_df = []

In [118]:
for i in range(16):
    file_path = f"../../register/data/people-{i}.csv"
    df = pd.read_csv(file_path)
    df = df.dropna(subset=['key_bbref', 'key_fangraphs'])
    df = df[['key_bbref', 'key_fangraphs']]
    df['key_fangraphs'] = df['key_fangraphs'].astype(int).astype(str)
    comparison_df.append(df)
comparison_df = pd.concat(comparison_df, ignore_index=True)

C:\Users\thoma\AppData\Local\Temp\ipykernel_20760\1510456781.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\thoma\AppData\Local\Temp\ipykernel_20760\1510456781.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\thoma\AppData\Local\Temp\ipykernel_20760\1510456781.py:3: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\thoma\AppData\Local\Temp\ipykernel_20760\1510456781.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\thoma\AppData\Local\Temp\ipykernel_20760\1510456781.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\th

In [119]:
# Merge the BBRef data with the comparison dataframe
df_merged = marcel_df.merge(comparison_df, on='key_bbref', how='left')

# Now you can use 'key_fg' instead of 'key_bbref'
df_merged.drop(columns=['key_bbref'], inplace=True)  # Optional: remove BBRef ID if not needed

df_merged = df_merged[df_merged['Season'] != 2025]

df_merged.rename(columns={df_merged.columns[-1]: 'IDfg'}, inplace=True)
df_merged = df_merged[['IDfg'] + list(df_merged.columns[:-1])]

In [120]:
df_merged = df_merged.rename(columns={'BA': 'AVG'})

In [121]:
actual = pd.read_csv('../Resources/all_data.csv')
actual = actual[['IDfg','Season', 'Name', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'SH', 'SB', 'wRC', 'WAR', 'G', 'AB', 'PA', 'AVG', 'BB%','OBP', 'SLG', 'OPS', 'ISO', 'wRAA', 'wRC+']]
actual = actual[(actual['Season'] >= 2022) & (actual['Season'] <= 2024)]
actual['IDfg'] = actual['IDfg'].astype(str)

C:\Users\thoma\AppData\Local\Temp\ipykernel_20760\238895524.py:1: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  actual = pd.read_csv('../Resources/all_data.csv')


In [122]:
columns_to_keep = ['IDfg','Season', 'Name','Team', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'SH', 'SB', 'wRC', 'WAR', 'G', 'AB', 'PA', 'AVG', 'BB%','OBP', 'SLG', 'OPS', 'ISO', 'wRAA', 'wRC+']
df_merged = df_merged[[col for col in columns_to_keep if col in df_merged.columns]]

In [123]:
df_merged = df_merged.rename(columns={col: f'Projected_{col}' for col in df_merged.columns if col not in ['IDfg','Name', 'Season', 'Team']})

In [124]:
marcel_df = pd.merge(df_merged, actual, on=['IDfg', 'Season'], how='inner')
marcel_df = marcel_df.drop(columns=['Name_y'])
marcel_df = marcel_df.rename(columns={'Name_x': 'Name'})

In [125]:
marcel_df

,IDfg,Season,Name,Projected_H,Projected_2B,Projected_3B,Projected_HR,Projected_R,Projected_RBI,Projected_BB,...,AB,PA,AVG,BB%,OBP,SLG,OPS,ISO,wRAA,wRC+
0,15676,2022,JosÃ© Abreu,142,29,1,28,77,102,45,...,601,679,0.304,0.091,0.378,0.446,0.824,0.141,27.8,137
1,18401,2022,Ronald Acuna Jr.,101,19,1,25,79,60,54,...,467,533,0.266,0.099,0.351,0.413,0.764,0.148,10.9,115
2,15986,2022,Willy Adames,123,28,1,21,69,61,50,...,563,617,0.238,0.079,0.298,0.458,0.756,0.220,7.3,109
3,19864,2022,Riley Adams,54,13,1,7,31,27,26,...,142,155,0.176,0.077,0.245,0.310,0.555,0.134,-7.5,58
4,20220,2022,Jo Adell,65,12,2,9,34,38,22,...,268,285,0.224,0.039,0.264,0.373,0.637,0.149,-7.2,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1392,14854,2024,Mike Yastrzemski,89,24,2,17,60,52,47,...,428,474,0.231,0.080,0.302,0.437,0.739,0.206,3.3,106
1393,11477,2024,Christian Yelich,131,26,2,15,89,63,73,...,270,315,0.315,0.127,0.406,0.504,0.909,0.189,20.6,153
1394,31837,2024,Masataka Yoshida,125,26,2,14,60,61,33,...,378,421,0.280,0.064,0.349,0.415,0.765,0.135,8.9,115
1395,29931,2024,Jacob Young,60,13,1,6,29,30,23,...,468,521,0.256,0.058,0.316,0.331,0.648,0.075,-8.6,85


In [126]:
marcel_df.columns

Index(['IDfg', 'Season', 'Name', 'Projected_H', 'Projected_2B', 'Projected_3B',
       'Projected_HR', 'Projected_R', 'Projected_RBI', 'Projected_BB',
       'Projected_SH', 'Projected_SB', 'Projected_AB', 'Projected_PA',
       'Projected_AVG', 'Projected_OBP', 'Projected_SLG', 'Projected_OPS', 'H',
       '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'SH', 'SB', 'wRC', 'WAR', 'G',
       'AB', 'PA', 'AVG', 'BB%', 'OBP', 'SLG', 'OPS', 'ISO', 'wRAA', 'wRC+'],
      dtype='object')

In [128]:
marcel_df.to_csv("../Projection_Results/marcel.csv", index=False)